## Game Data

In [ ]:
# !pip install chessdata

In [ ]:
from pathlib import Path
from chessdata.etf import pgn2df

In [ ]:
path = Path("./")
pgns = Path(path/"pgns")
players = [pgn.stem for pgn in pgns.glob("*.pgn")]
players

Extract the game metadata from the pgn into a csv.

In [ ]:
for player in players:
    pgn = open(pgns/f"{player}.pgn")
    df = pgn2df(pgn)
    df = df[df['Date'] > '2020.01.01'] ## only keep games after 2020
    df = df[df['Site'].str.contains(".com")==False] ## only keep over-the-board chess (i.e. remove online games)
    df = df[df['Site'].str.contains(".org")==False] ## only keep over-the-board chess (i.e. remove online games)
    df = df[df['Event'].str.contains("Online")==False] ## only keep over-the-board chess (i.e. remove online games)
    print(player)
    print(df.shape)
    df.to_csv(path/f"output/metadata/{player}.csv")

## Engine Evals

In [ ]:
from datetime import datetime
import chess.engine
from chessdata.engine import evaluate_pgn

In [ ]:
stockfish = './stockfish_15_x64_avx2.exe' # location of your stockfish executable

Analyze all games move-by-move. Warning: this takes a long time!

In [ ]:
for player in players:
    print(player)
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    engine = chess.engine.SimpleEngine.popen_uci(stockfish)
    pgn = open(pgns/f"{player}.pgn")
    evals = evaluate_pgn(pgn, engine, limit=chess.engine.Limit(depth=15))
    evals.to_csv(path/f"output/centipawns/{player}.csv")